In [1]:
from osgeo import gdal
from qgis.core import QgsApplication, QgsCoordinateReferenceSystem
import numpy as np
import sys, os
import matplotlib.pyplot as plt

# Initiating a QGIS application and connect to processing
qgishome = 'C:/OSGeo4W/apps/qgis/'
QgsApplication.setPrefixPath(qgishome, True)
app = QgsApplication([], False)

app.initQgis()

windowsuser = 'xbacos'
sys.path.append('C:/OSGeo4W/apps/qgis/python/plugins')
sys.path.append('C:/Users/' + windowsuser + '/AppData/Roaming/QGIS/QGIS3/profiles/default/python/plugins') # path to third party plugins
sys.path.append('C:/FUSION')

import processing
from processing.core.Processing import Processing
Processing.initialize()

In [2]:
# Uppdatera till dina filer
indir = 'C:/temp/dsms/'
outdir = indir + 'out/'
dsm_in = indir + 'DSM_flagstaff_reproj.tif'

dsm = outdir + 'dsm.tif'
dem = outdir + 'dem.tif'
cdsm = outdir + 'cdsm.tif'
lc = outdir + 'lc.tif'
building = indir +  'buildings_reproj.shp'
trees =  indir + 'trees_flagstaff_reproj.shp'

crs_in = 'ESPG:32755'            # Uppdatera till ditt koordinatsysyem
crs = 'ESPG:32755'               # Uppdatera till ditt koordinatsystem

output = {}

In [10]:
# Skapa nya rasters, reproject och gör om till 1m
for i in [dem, dsm, cdsm, lc]:
    processing.run("gdal:warpreproject", {
            'INPUT': dsm_in,
            'SOURCE_CRS':QgsCoordinateReferenceSystem(crs_in),
            'TARGET_CRS':QgsCoordinateReferenceSystem(crs),
            'RESAMPLING':0,
            'NODATA':None,
            'TARGET_RESOLUTION':1,
            'OPTIONS':'',
            'DATA_TYPE':0,
            'TARGET_EXTENT':None,
            'TARGET_EXTENT_CRS':QgsCoordinateReferenceSystem(crs),
            'MULTITHREADING':False,
            'EXTRA':'',
            'OUTPUT': i
            }
            )
            
# Buffra byggnader och trädpolygon
# Träd 
buffersize = 1 # Sätt var du tycker är rimligt

# tree_buffer = outdir + 'tree_buff.shp'
output['treebuff'] = processing.run("native:buffer", {
    'INPUT': trees,
    'DISTANCE':buffersize,
    'SEGMENTS':5,'END_CAP_STYLE':0,'JOIN_STYLE':0,'MITER_LIMIT':2,
    'DISSOLVE':True,
    'OUTPUT': 'TEMPORARY_OUTPUT'})

# Buildings
buffersize = 2 # Sätt var du tycker är rimligt

#= build_buffer = outdir + 'build_buff.shp'
output['buildbuff']  = processing.run("native:buffer", {
    'INPUT': building,
    'DISTANCE':buffersize,
    'SEGMENTS':5,'END_CAP_STYLE':0,'JOIN_STYLE':0,'MITER_LIMIT':2,
    'DISSOLVE':True,
    'OUTPUT':'TEMPORARY_OUTPUT'})
    
# Merge buffers
#= merge_layer = outdir + 'merged.shp'
output['merge_buff']  = processing.run("native:mergevectorlayers", {
    'LAYERS':[output['treebuff']['OUTPUT'], output['buildbuff']['OUTPUT']],
    'CRS':QgsCoordinateReferenceSystem(crs),
    'OUTPUT': 'TEMPORARY_OUTPUT'})

output['bbox'] = processing.run("native:polygonfromlayerextent", {
    'INPUT': dsm,
    'ROUND_TO':0,
    'OUTPUT':'TEMPORARY_OUTPUT'})

output['difference'] = processing.run("native:difference", {
    'INPUT':output['bbox']['OUTPUT'],
    'OVERLAY': building,
    'OUTPUT':'TEMPORARY_OUTPUT',
    'GRID_SIZE':None})


In [4]:
# DEM Sätt nan där det är byggnad eller träd
processing.run("gdal:rasterize_over_fixed_value", {
    'INPUT':output['merge_buff']['OUTPUT'],
    'INPUT_RASTER':dem,
    'BURN':np.nan,'ADD':False,'EXTRA':''})

# Interpolera
dem_out = outdir + 'dem_out.tif'
processing.run("grass7:r.fillnulls", {
'input': dem,
'method':2,'tension':40,'smooth':0.1,'edge':3,'npmin':500,'segmax':300,
'lambda':0.01,
'output': dem_out,
'GRASS_REGION_PARAMETER':None,'GRASS_REGION_CELLSIZE_PARAMETER':0,'GRASS_RASTER_FORMAT_OPT':'','GRASS_RASTER_FORMAT_META':''})


{'output': 'C:\\temp\\dsms\\out\\dem_out.tif'}

In [6]:
# DsM Sätt nan där det är träd
processing.run("gdal:rasterize_over_fixed_value", {
    'INPUT': output['difference']['OUTPUT'],
    'INPUT_RASTER':dsm,
    'BURN': 0,'ADD':False,
    'EXTRA':''})

dsm_out = outdir + 'dsm_out.tif'
processing.run("gdal:rastercalculator", {
    'INPUT_A':dem_out,'BAND_A':1,
    'INPUT_B':dsm,'BAND_B':None,
    'INPUT_C':None,'BAND_C':None,'INPUT_D':None,'BAND_D':None,'INPUT_E':None,'BAND_E':None,'INPUT_F':None,'BAND_F':None,'FORMULA':'A+B','NO_DATA':None,'PROJWIN':None,'RTYPE':5,'OPTIONS':'','EXTRA':'',
    'OUTPUT': dsm_out})


{'OUTPUT': 'C:/temp/dsms/out/dsm_out.tif'}

In [24]:
# # CDSM Sätt nan där det är byggnader
processing.run("gdal:rasterize_over_fixed_value", {
    'INPUT': output['buildbuff']['OUTPUT'],
    'INPUT_RASTER':cdsm,
    'BURN':np.nan,'ADD':False,'EXTRA':''})

cdsm_norm = outdir + 'cdsm_norm.tif'
output['CDSM_norm'] = processing.run("gdal:rastercalculator", {
    'INPUT_A': cdsm,'BAND_A':1,
    'INPUT_B':dem_out,'BAND_B'
    :None,'INPUT_C':None,'BAND_C':None,'INPUT_D':None,'BAND_D':None,'INPUT_E':None,'BAND_E':None,'INPUT_F':None,'BAND_F':None,'FORMULA':'A-B','NO_DATA':None,'PROJWIN':None,'RTYPE':5,'OPTIONS':'','EXTRA':'',
    'OUTPUT': cdsm_norm})

processing.run("gdal:rasterize_over_fixed_value", {
    'INPUT': output['buildbuff']['OUTPUT'],
    'INPUT_RASTER': output['CDSM_norm']['OUTPUT'],
    'BURN':np.nan,'ADD':False,'EXTRA':''})

output['CDSM_filt'] = processing.run("gdal:rastercalculator", {'INPUT_A':output['CDSM_norm']['OUTPUT'],'BAND_A':1,
    'INPUT_B':dem_out,'BAND_B':1,'INPUT_C':None,'BAND_C':None,'INPUT_D':None,'BAND_D':None,'INPUT_E':None,'BAND_E':None,'INPUT_F':None,'BAND_F':None,
    'FORMULA':'(A > 0.5)* A','NO_DATA':None,'PROJWIN':None,'RTYPE':5,'OPTIONS':'','EXTRA':'','OUTPUT': 'TEMPORARY_OUTPUT'})

# Fill nodata med 0
cdsm_out = outdir + 'cdsm_out.tif'
processing.run("native:fillnodata", {
'INPUT': output['CDSM_filt']['OUTPUT'],
'BAND':1,
'FILL_VALUE':0,
'OUTPUT': cdsm_out})


{'OUTPUT': 'C:/temp/dsms/out/cdsm_out.tif'}

In [9]:
# Landcover 
lc_bool = outdir + 'lc_bool.tif'
processing.run("gdal:rastercalculator", {'INPUT_A':lc,'BAND_A':1,
    'INPUT_B':None,'BAND_B':None,'INPUT_C':None,'BAND_C':None,'INPUT_D':None,'BAND_D':None,'INPUT_E':None,'BAND_E':None,'INPUT_F':None,'BAND_F':None,
    'FORMULA':'(A * 0) + 1','NO_DATA':None,'PROJWIN':None,'RTYPE':5,'OPTIONS':'','EXTRA':'','OUTPUT': lc_bool})


{'OUTPUT': 'C:/temp/dsms/out/lc_bool.tif'}